In [1]:
using CSV, Dates, JuliaDB

In [2]:
ionex_file = "../jplg3660.12i"

"../jplg3660.12i"

In [3]:
# OPen the file and place its contents in an array name lines
file = open(ionex_file);
lines = readlines(file)
close(file)

In [4]:
using Dates
index = 0
tec_values = []   # Tec values per longitud, lat(-180,180)
tec = []          # All the tec values in a file
dates_array = []  # Store the different time stamps

for line in lines
    index += 1
    
    # Obtain time_stamp and tec values for every map (13 TEC maps in a file)
    if occursin(r"START OF TEC MAP", line)
        date_line = lines[index+1]
        date_units = []
        string = split(date_line)
        
        for x in 1:6
            push!(date_units,parse(Int,string[x]))
        end
        
        # Date units 
        year = date_units[1]
        month = date_units[2]
        day = date_units[3]
        hour = date_units[4]
        minutes = date_units[5]
        seconds = date_units[6]
        
        # Map time_stamp for a period(2 hours)
        time_stamp = DateTime(year, month, day, hour, minutes, seconds)
        push!(dates_array, time_stamp)
    end
    
    # Tec values per latitud in the longitud range (-180,180)
    if occursin(r"LAT/LON1/LON2/DLON/H", line)
        for x in 1:5
            tec_values = split(lines[index + x])
            for n in tec_values
                push!(tec, n)
            end
        end
    end
    
end

In [7]:
# used to create a latitud, to hold 2-hour-interval Tec map data
function create_coordinates(start, stop, step)
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:73
            push!(coordinates,i)
        end
    end
    coordinates # Julia automatically returns the last statement
end
    

create_coordinates (generic function with 1 method)

In [ ]:
latitud = create_coordinates(87.5, -87.5, -2.5)
longitud = collect(Iterators.flatten([collect(-180:5:180) for i in 1:71])) # I Still don't grok this!

In [5]:
for d in dates_array
    print(d,'\n')
end

2012-12-31T00:00:00
2012-12-31T02:00:00
2012-12-31T04:00:00
2012-12-31T06:00:00
2012-12-31T08:00:00
2012-12-31T10:00:00
2012-12-31T12:00:00
2012-12-31T14:00:00
2012-12-31T16:00:00
2012-12-31T18:00:00
2012-12-31T20:00:00
2012-12-31T22:00:00
2013-01-01T00:00:00


In [9]:
time_stamp = [dates_array[1] for i in 1:5183]

5183-element Array{DateTime,1}:
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 ⋮                  
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00
 2012-12-31T00:00:00

In [10]:
# Convert string values to integers for storing usable values in CSV
depth = [parse(Int,t) for t in tec[1:5183]]

5183-element Array{Int64,1}:
  38
  38
  39
  39
  39
  39
  39
  39
  39
  39
  39
  39
  39
   ⋮
 174
 174
 174
 175
 175
 175
 175
 175
 175
 175
 176
 176

In [11]:
t = table((time_stamp=time_stamp, latitud=latitud, longitud=longitud, depth=depth); pkey = [:time_stamp, :latitud, :longitud, :depth])

Table with 5183 rows, 4 columns:
time_stamp           latitud  longitud  depth
─────────────────────────────────────────────
2012-12-31T00:00:00  -87.5    -180      176
2012-12-31T00:00:00  -87.5    -175      176
2012-12-31T00:00:00  -87.5    -170      176
2012-12-31T00:00:00  -87.5    -165      176
2012-12-31T00:00:00  -87.5    -160      176
2012-12-31T00:00:00  -87.5    -155      176
2012-12-31T00:00:00  -87.5    -150      176
2012-12-31T00:00:00  -87.5    -145      175
2012-12-31T00:00:00  -87.5    -140      175
2012-12-31T00:00:00  -87.5    -135      175
2012-12-31T00:00:00  -87.5    -130      175
2012-12-31T00:00:00  -87.5    -125      175
⋮
2012-12-31T00:00:00  87.5     130       36
2012-12-31T00:00:00  87.5     135       36
2012-12-31T00:00:00  87.5     140       36
2012-12-31T00:00:00  87.5     145       36
2012-12-31T00:00:00  87.5     150       36
2012-12-31T00:00:00  87.5     155       37
2012-12-31T00:00:00  87.5     160       37
2012-12-31T00:00:00  87.5     165       37
2

In [13]:
CSV.write("julia.csv", t)

"julia.csv"

In [14]:
length(tec)/5183

26.0